In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import pandas as pd
from joblib import dump


In [2]:
file = "/Users/liam/PycharmProjects/dataspell/artemis/data/processed/whois_data.csv"
df = pd.read_csv(file)
features = ['dns_rec_mx_cc', 'entropy', 'creation_date', 'days_since_creation']


/Users/liam/miniconda3/envs/artemis/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (48,99,100,101,102,103,104,108,109,110,111,112,113,115,116,117,118,119,120,121,122,126,132,133,136,138,139,140,141,144,145,146,147,149,150,151,152,156,160,161,163,164,165,166,167,168) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
label = 'malicious'
labels = df[label].values
data_df = df[features]
#data_df = df.drop(columns=label)

In [4]:
data_df['creation_date'] = pd.to_datetime(data_df.creation_date)
data_df['year'] = data_df.creation_date.dt.year
data_df['month'] = data_df.creation_date.dt.month
data_df['day'] = data_df.creation_date.dt.day
data_df = data_df.drop(columns='creation_date')
data_df = data_df.fillna(-1)

/var/folders/s4/04m6jmt93ll3w72t198n3k3c0000gn/T/ipykernel_49521/286784156.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df['creation_date'] = pd.to_datetime(data_df.creation_date)
/var/folders/s4/04m6jmt93ll3w72t198n3k3c0000gn/T/ipykernel_49521/286784156.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df['year'] = data_df.creation_date.dt.year
/var/folders/s4/04m6jmt93ll3w72t198n3k3c0000gn/T/ipykernel_49521/286784156.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [5]:
X_train, X_test, y_train, y_test = train_test_split(data_df, labels, random_state=0)

In [6]:
labelenc = LabelEncoder().fit(data_df[features[0]])
X_train[features[0]+"_enc"] = labelenc.transform(X_train[features[0]])
X_train = X_train.drop(columns=features[0])
X_test[features[0]+"_enc"] = labelenc.transform(X_test[features[0]])
X_test = X_test.drop(columns=features[0])


In [7]:
data_df

,dns_rec_mx_cc,entropy,days_since_creation,year,month,day
0,us,1.918296,8960.0,1997.0,9.0,15.0
1,us,2.750000,9130.0,1997.0,3.0,29.0
2,us,2.521641,6250.0,2005.0,2.0,15.0
3,us,2.128085,8102.0,2000.0,1.0,21.0
4,us,2.947703,6506.0,2004.0,6.0,4.0
...,...,...,...,...,...,...
6505,na,3.000000,252.0,2021.0,7.0,19.0
6506,na,3.000000,252.0,2021.0,7.0,19.0
6507,nl,3.378783,3272.0,2013.0,4.0,12.0
6508,us,3.080501,2073.0,2016.0,7.0,24.0


In [8]:
clf = RandomForestClassifier(criterion='entropy', random_state=0).fit(X_train.values, y_train)

In [9]:
clf.score(X_test, y_test)

/Users/liam/miniconda3/envs/artemis/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


0.9778869778869779

In [10]:
model_loc = "../models/"
model_name = "rfc.joblib"
enc_name = "enc.joblib"

In [11]:
dump(clf, model_loc+model_name)

['../models/rfc.joblib']

In [12]:
dump(labelenc, model_loc+enc_name)

['../models/enc.joblib']

In [14]:
clf.feature_importances_

array([0.13983754, 0.30454804, 0.30823104, 0.0337287 , 0.04418349,
       0.16947118])

In [20]:
X_train

,entropy,days_since_creation,year,month,day,dns_rec_mx_cc_enc
5528,3.642490,289.0,2021.0,6.0,12.0,31
3424,2.321928,8691.0,1998.0,6.0,11.0,49
4977,2.845351,-1.0,-1.0,-1.0,-1.0,31
5056,3.327820,147.0,2021.0,11.0,1.0,31
3008,1.584963,-1.0,-1.0,-1.0,-1.0,12
...,...,...,...,...,...,...
4931,3.189898,-1.0,-1.0,-1.0,-1.0,31
3264,3.323074,4507.0,2009.0,11.0,24.0,49
1653,2.921928,9824.0,1995.0,5.0,5.0,49
2607,0.000000,7284.0,2002.0,4.0,18.0,31
